# ⚡ ULTRA FAST Video Upscale

**Tối ưu TỐC ĐỘ TỐI ĐA - chất lượng vừa đủ**

### ⏱️ Thời gian:
- Phim 2h: **~4-6 giờ** (thay vì 30-40h)
- Phim 1h: **~2-3 giờ**

### 🔄 Resume:
- Nếu ngắt giữa chừng → chạy lại Cell 4+5
- Tự động skip video đã xong
- Progress lưu vào Drive

In [ ]:
#@title 1️⃣ Mount Drive
from google.colab import drive
import os, glob, subprocess, json, time, pickle
from datetime import timedelta

drive.mount('/content/drive')
!nvidia-smi --query-gpu=name,memory.free --format=csv

In [ ]:
#@title 2️⃣ Install (chỉ lần đầu)
if not os.path.exists('/content/Real-ESRGAN/weights/realesr-animevideov3.pth'):
    !git clone https://github.com/xinntao/Real-ESRGAN.git
    %cd /content/Real-ESRGAN
    !pip install basicsr facexlib gfpgan -q
    !pip install -r requirements.txt -q
    !python setup.py develop 2>&1 | tail -1
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth -P weights/ -q
print('✅ Ready!')

In [ ]:
#@title 3️⃣ Cấu hình
#@markdown ### 📂 Đường dẫn
INPUT_DIR = "/content/drive/MyDrive/Movies/Input" #@param {type:"string"}
OUTPUT_DIR = "/content/drive/MyDrive/Movies/Output" #@param {type:"string"}

#@markdown ---
#@markdown ### 🚀 Chạy Multi-Colab (song song)
#@markdown Nếu bạn mở nhiều tab Colab, mỗi tab chạy 1 phần:
WORKER_ID = 1 #@param {type:"integer"}
TOTAL_WORKERS = 1 #@param {type:"integer"}
#@markdown > VD: 3 phim, 2 Colab → Tab1: WORKER=1, Tab2: WORKER=2

# Progress file (để resume)
PROGRESS_FILE = f"/content/drive/MyDrive/Movies/.progress_worker{WORKER_ID}.pkl"

# ULTRA FAST settings
TILE = 64          # Nhỏ nhất = nhanh nhất
CRF = 28           # Cao = file nhỏ, encode nhanh
PRESET = 'ultrafast'  # Nhanh nhất

print(f"⚡ ULTRA FAST MODE")
print(f"📂 Input: {INPUT_DIR}")
print(f"📂 Output: {OUTPUT_DIR}")
print(f"🔧 Tile={TILE}, CRF={CRF}, Preset={PRESET}")
print(f"👷 Worker {WORKER_ID}/{TOTAL_WORKERS}")

In [ ]:
#@title 4️⃣ Scan & Phân chia video
def get_info(p):
    try:
        r = subprocess.run(['ffprobe','-v','quiet','-print_format','json','-show_format','-show_streams',p], capture_output=True, text=True, timeout=30)
        d = json.loads(r.stdout)
        v = next((s for s in d.get('streams',[]) if s.get('codec_type')=='video'), {})
        return {'w': int(v.get('width',0)), 'h': int(v.get('height',0)), 'dur': float(d.get('format',{}).get('duration',0))}
    except: return {'w':0,'h':0,'dur':0}

# Load progress đã hoàn thành
done = set()
if os.path.exists(PROGRESS_FILE):
    with open(PROGRESS_FILE, 'rb') as f:
        done = pickle.load(f)
    print(f"📂 Đã load {len(done)} video hoàn thành từ lần trước")

# Scan tất cả video
all_vids = []
for ext in ('*.mp4','*.mkv','*.avi','*.mov','*.webm'):
    all_vids.extend(glob.glob(os.path.join(INPUT_DIR,'**',ext), recursive=True))
all_vids = sorted(all_vids)

# Phân chia cho worker này
my_vids = [v for i,v in enumerate(all_vids) if (i % TOTAL_WORKERS) == (WORKER_ID - 1)]

# Lọc ra những cái chưa xong
to_process = []
for p in my_vids:
    name = os.path.basename(p)
    out = os.path.join(OUTPUT_DIR, name.rsplit('.',1)[0] + '_4K.mkv')
    info = get_info(p)
    
    if name in done:
        print(f"⏭️ [RESUME-SKIP] {name}")
    elif os.path.exists(out) and os.path.getsize(out) > 1024*1024:
        print(f"⏭️ [DONE] {name}")
        done.add(name)
    elif info['w'] >= 3800:
        print(f"⏭️ [4K] {name}")
    else:
        to_process.append({'path': p, 'name': name, 'info': info})
        dur = timedelta(seconds=int(info['dur']))
        print(f"✅ [QUEUE] {name} ({info['w']}x{info['h']}, {dur})")

# Summary
total_dur = sum(v['info']['dur'] for v in to_process)
est_hours = (total_dur / 3600) * 2.5  # ~2.5h xử lý cho 1h video
print(f"\n📊 Worker {WORKER_ID}: {len(to_process)} videos | ETA: ~{est_hours:.1f}h")
if TOTAL_WORKERS > 1:
    print(f"💡 Tổng {len(all_vids)} videos chia cho {TOTAL_WORKERS} workers")

In [ ]:
#@title 5️⃣ 🚀 CHẠY!
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs('/content/temp', exist_ok=True)

for idx, item in enumerate(to_process, 1):
    p, name, info = item['path'], item['name'], item['info']
    out = os.path.join(OUTPUT_DIR, name.rsplit('.',1)[0] + '_4K.mkv')
    temp = f'/content/temp/{idx}.mp4'
    
    print(f"\n{'='*50}")
    print(f"⚡ [{idx}/{len(to_process)}] {name}")
    print(f"{'='*50}")
    start = time.time()
    
    try:
        # UPSCALE (FAST)
        print("🎨 Upscaling...")
        %cd /content/Real-ESRGAN
        !python inference_realesrgan_video.py \
            -n realesr-animevideov3 \
            -i "{p}" -o "{temp}" -s 2 \
            --tile {TILE} --tile_pad 8 \
            --suffix "" --fp32
        
        if not os.path.exists(temp):
            raise Exception("Upscale failed")
        
        # ENCODE (FAST)
        print("📦 Encoding...")
        !ffmpeg -y -i "{p}" -i "{temp}" \
            -map 1:v -map 0:a? -map 0:s? \
            -c:v libx264 -crf {CRF} -preset {PRESET} \
            -c:a copy -c:s copy \
            -map_metadata 0 -map_chapters 0 \
            "{out}" -loglevel warning -stats
        
        # Save progress
        done.add(name)
        with open(PROGRESS_FILE, 'wb') as f:
            pickle.dump(done, f)
        
        elapsed = time.time() - start
        size = os.path.getsize(out) / (1024**3)
        print(f"✅ Done: {timedelta(seconds=int(elapsed))} | {size:.2f}GB")
        
    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        if os.path.exists(temp): os.remove(temp)

print(f"\n🏁 Worker {WORKER_ID} HOÀN THÀNH!")
print(f"📂 Output: {OUTPUT_DIR}")

---
## 📖 Hướng dẫn

### 🔄 Resume nếu bị ngắt:
1. Mở lại Colab
2. Chạy Cell 1 → 2 → 3 → 4 → 5
3. Tự động skip video đã xong!

### 🚀 Chạy Multi-Colab (nhanh gấp đôi):
1. Mở **2 tab** Colab với cùng notebook
2. **Tab 1:** `WORKER_ID = 1`, `TOTAL_WORKERS = 2`
3. **Tab 2:** `WORKER_ID = 2`, `TOTAL_WORKERS = 2`
4. Chạy cả 2 → mỗi tab xử lý nửa số video

### 💡 Keep Alive:
Paste vào Console (F12) để không bị ngắt:
```javascript
setInterval(()=>document.querySelector('colab-connect-button')?.click(), 60000)
```